### **사이트 내 봇탐지 기능으로 인해 중단 시, 크롤링이 중단된 지점에서부터 다시 실행**

In [1]:
!pip install fake-useragent
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import random
from collections import OrderedDict
from requests.exceptions import RequestException
from fake_useragent import UserAgent

# ✅ 브라우저처럼 보이게 User-Agent 풀
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
]

# ✅ 도 이름 축약 함수
def simplify_address(address: str) -> str:
    replacements = {
        "경상북도": "경북", "경상남도": "경남",
        "전라북도": "전북", "전라남도": "전남",
        "충청북도": "충북", "충청남도": "충남",
        "강원도": "강원", "경기도": "경기",
        "제주특별자치도": "제주", "전북특별자치도": "전북",
        "서울특별시": "서울","부산광역시": "부산",
        "대구광역시": "대구","인천광역시": "인천",
        "광주광역시": "광주","대전광역시": "대전",
        "울산광역시": "울산","세종특별자치시": "세종",
    }
    for full, short in replacements.items():
        address = address.replace(full, short)
    return address

# ✅ 동 정보를 괄호로 추가
def enrich_address_with_dong(full_address: str) -> str:
    lines = full_address.split("\n")
    if len(lines) < 2:
        return full_address.strip()
    first_line = lines[0].strip()
    second_line = lines[1].strip()
    match = re.search(r"(시|군|구)\s+([가-힣]+동)", second_line)
    if match:
        dong = match.group(2)
        return f"{first_line} ({dong})"
    else:
        return first_line

# ✅ 안전 요청 함수 (재시도 포함)
def safe_request(url, headers, proxies=None, retries=3):
    for i in range(retries):
        try:
            res = requests.get(url, headers=headers, proxies=proxies, timeout=60)
            res.raise_for_status()
            return res
        except RequestException:
            print(f"⏳ 재시도 {i+1}/{retries}...")
            time.sleep(random.uniform(2, 4))
    return None

# ✅ 업체 단위 크롤링 함수
def get_bizno_info(company_name: str):
    # ✅ 다층 사용자 정의 User-Agent (OS + 브라우저 랜덤 구성)
    def generate_custom_user_agent():
        os_list = [
            "Windows NT 10.0; Win64; x64",
            "Macintosh; Intel Mac OS X 13_4_1",
            "X11; Linux x86_64",
            "Linux; Android 12; SM-G991B",
            "iPhone; CPU iPhone OS 16_3 like Mac OS X"
        ]

        browsers = [
            lambda os: f"Mozilla/5.0 ({os}) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(120, 125)}.0.{random.randint(4000,6000)}.100 Safari/537.36",
            lambda os: f"Mozilla/5.0 ({os}) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15",
            lambda os: f"Mozilla/5.0 ({os}) Gecko/20100101 Firefox/{random.randint(120, 126)}.0",
            lambda os: f"Mozilla/5.0 ({os}) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.94 Safari/537.36 Edg/122.0.2365.66",
        ]

        os = random.choice(os_list)
        browser = random.choice(browsers)
        return browser(os)

    # ✅ fake-useragent + fallback으로 자동 전환되는 UA 생성기
    def get_user_agent():
        try:
            from fake_useragent import UserAgent
            ua = UserAgent()
            user_agent = ua.random
            print("✅ fake-useragent 사용")
        except Exception as e:
            print(f"⚠️ fake-useragent 실패 → 커스텀 User-Agent 사용: {e}")
            user_agent = generate_custom_user_agent()
        return user_agent

    headers = {
        "User-Agent": get_user_agent(),
        "Referer": "https://bizno.net/",
        "Accept-Language": random.choice(["ko-KR,ko;q=0.9", "en-US,en;q=0.8", "ja-JP,ja;q=0.9"]),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    proxies = [
        "http://138.68.60.8:80",
        "http://101.255.211.22:8181",
        "http://102.213.241.70:8080",
    ]

    proxy = {"http": random.choice(proxies), "https": random.choice(proxies)}



    search_url = f"https://bizno.net/?query={company_name}"
    res = safe_request(search_url, headers, None)
    if not res:
        return {"업체명": company_name, "오류": "검색 페이지 요청 실패"}

    soup = BeautifulSoup(res.text, "html.parser")
    # 첫 번째 검색 결과 링크를 바로 선택
    a_tag = soup.select_one("section a[href^='/article/']")
    if not a_tag:
        print(f"[{company_name}] 검색 결과 없음")
        return {"업체명": company_name, "오류": "검색 결과 없음"}

    # ✅ 표시명 추출
    h4_tag = a_tag.find("h4")
    displayed_name = h4_tag.text.strip() if h4_tag else ""

    # 상세 페이지 요청
    detail_url = "https://bizno.net" + a_tag['href']
    res = safe_request(detail_url, headers, None)
    # ... 이후 기존 코드 유지 ...

    detail_url = "https://bizno.net" + a_tag['href']
    res = safe_request(detail_url, headers, None)
    if not res:
        return {"업체명": company_name, "오류": "상세 페이지 요청 실패"}

    soup = BeautifulSoup(res.text, "html.parser")
    tbody = soup.find("body")
    if not tbody:
        return {"업체명": company_name, "오류": "페이지 구조 오류"}

    data = {}
    for row in tbody.find_all("tr"):
        th = row.find("th")
        td = row.find("td")
        if not th or not td:
            continue
        key = th.get_text(strip=True)
        value = td.get_text("\n", strip=True)
        if key == "회사주소":
            value = enrich_address_with_dong(value)
            value = simplify_address(value)
        data[key] = value

    wanted_keys = [
        "회사주소", "주요제품", "대표자명", "법인구분", "사업자등록번호",
        "법인등록번호", "홈페이지", "전화번호", "종목", "국세청산업분류", "업태"
    ]

    # ✅ 결과에 표시명 추가
    filtered_data = OrderedDict()
    filtered_data["원래표시명"] = displayed_name
    filtered_data["업체명"] = company_name
    for k in wanted_keys:
        filtered_data[k] = data.get(k, "")

    return filtered_data

# ✅ 전체 Excel 처리
def crawl_from_excel(input_path: str, output_path: str, start_company=None):
    df = pd.read_excel(input_path)

    # ✅ 업체명과 사업자등록번호 모두 불러오기
    if "업체명" not in df.columns or "사업자등록번호" not in df.columns:
        raise ValueError("엑셀에 '업체명' 또는 '사업자등록번호' 열이 없습니다.")

    df = df[["업체명", "사업자등록번호"]].dropna(subset=["업체명"])
    rows = df.to_dict("records")

    # ✅ 시작 업체명 필터링
    if start_company:
        start_index = next((i for i, row in enumerate(rows) if row["업체명"] == start_company), None)
        if start_index is not None:
            rows = rows[start_index:]
            print(f"🔁 '{start_company}'부터 크롤링 시작합니다 (index={start_index})")
        else:
            print(f"❗ 시작 업체명 '{start_company}'을 찾을 수 없습니다. 전체에서 시작합니다.")

    results, failed = [], []

    for idx, row in enumerate(rows, 1):
        name = row["업체명"]
        bizno = str(row.get("사업자등록번호", "")).strip()

        query = bizno if bizno and bizno != "nan" else name
        print(f"[{idx}/{len(rows)}] 크롤링 중: {query} ({'사업자번호' if query == bizno else '업체명'})")

        info = get_bizno_info(query)

        # ✅ 업체명 조회 여부 기록
        if query == name:
            info["업체명"] = f"(업체명검색) {name}"
        else:
            info["업체명"] = name

        if "오류" in info:
            if info["오류"] == "검색 결과 없음":
                print("🛑 검색 결과 없음 → 크롤링 중단 및 저장")
                failed.append(info)
                break
            failed.append(info)
        else:
            results.append(info)

        time.sleep(random.uniform(0.5, 1))
        if idx % 20 == 0:
            print("💤 서버 보호용 대기 중...")
            time.sleep(1)

    all_results = results + failed
    df_final = pd.DataFrame(all_results)
    df_final.to_excel(output_path, index=False)
    print(f"\n✅ 크롤링 완료! 결과 저장: {output_path}")
    print(f"❗ 실패한 업체 수: {len(failed)}")



# ✅ 실행, 중단 시 중단 지점의 이름을 예시와 같이 입력 후 재실행 ( , start_company="전북도의회" )
if __name__ == "__main__":
    crawl_from_excel("기업지원관리_직무역량 (6번_김동혁).xlsx", "(1차)크롤링_결과(최종).xlsx")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 7.1 MB/s eta 0:00:00
[1/1978] 크롤링 중: J000006078 (사업자번호)
✅ fake-useragent 사용
[2/1978] 크롤링 중: 1058132709 (사업자번호)
✅ fake-useragent 사용
[3/1978] 크롤링 중: 0000000000 (사업자번호)
✅ fake-useragent 사용
[4/1978] 크롤링 중: 0000000000 (사업자번호)
✅ fake-useragent 사용
[5/1978] 크롤링 중: 0000000000 (사업자번호)
✅ fake-useragent 사용
[6/1978] 크롤링 중: 4188208456 (사업자번호)
✅ fake-useragent 사용
[7/1978] 크롤링 중: 0000000000 (사업자번호)
✅ fake-useragent 사용
[8/1978] 크롤링 중: 4028284027 (사업자번호)
✅ fake-useragent 사용
[9/1978] 크롤링 중: 4028213594 (사업자번호)
✅ fake-useragent 사용
[10/1978] 크롤링 중: 4028220019 (사업자번호)
✅ fake-useragent 사용
[11/1978] 크롤링 중: 4028218570 (사업자번호)
✅ fake-useragent 사용
[12/1978] 크롤링 중: 2148204203 (사업자번호)
✅ fake-useragent 사용
[13/1978] 크롤링 중: 2208205283 (사업자번호)
✅ fake-useragent 사용
[14/1978] 크롤링 중: (유)가나안 (업체명)
✅ fake-useragent 사용
[15/1978] 크롤링 중: 8758800149 (사업자번호)
✅ fake-useragent 사용
[16/1978] 크롤링 중: 4028600684 (사업자번호)
✅ fake-useragent 사용
[17/1978] 크롤링 중: (유)남원조립식건축 (업체명)
✅ fak

KeyboardInterrupt: 

In [ ]:
# 업체 단일 조회
result = get_bizno_info("(재)예수병원유지재단")
pd.set_option('display.max_colwidth', None)

# DataFrame으로 변환
if isinstance(result, dict):
    test_df = pd.DataFrame([result])  # ✅ 리스트로 감싸서 DataFrame 생성
    display(test_df)  # 주피터/코랩에선 display() 추천
else:
    print("❗ 크롤링 결과 없음 또는 오류 발생")


✅ fake-useragent 사용


,원래표시명,업체명,회사주소,주요제품,대표자명,법인구분,사업자등록번호,법인등록번호,홈페이지,전화번호,종목,국세청산업분류,업태
0,(재)예수병원유지재단,(재)예수병원유지재단,전북 전주시 완산구 서원로 365(중화산동1가) (중화산동),의료서비스,임기수,법인기업,402-82-00712,210133-0000443,jesushospital.com,063-230-8561,종합병원,대분류 : 보건업 및 사회복지 서비스업\n중분류 : 보건업\n소분류 : 병원\n세분류 : 병원\n세세분류 : 종합병원,의료업


### **크롤링 파일 통합 후, 세세분류 정제**  ↓

In [2]:
import pandas as pd

# 엑셀 파일 경로
file_path = "/content/test통합 문서1.xlsx"  # Colab에 업로드한 경로에 맞게 수정

# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

# 세세분류 이후 텍스트만 추출
def extract_after_subcategory(text):
    if isinstance(text, str) and "세세분류 " in text:
        return text.split("세세분류 ")[-1].strip()
    return ""

# 해당 열 정제
df["국세청산업분류(한국표준산업분류 참조)"] = df["국세청산업분류(한국표준산업분류 참조)"].apply(extract_after_subcategory)

# ✅ 결과 출력
df.to_excel("전처리_test.xlsx", index=False)  # 또는 그냥 df 로도 됨
